# FEAX Quick Start in Google Colab

This notebook demonstrates how to install and use FEAX (Finite Element Analysis with JAX) in Google Colab.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Naruki-Ichihara/feax/blob/main/examples/colab_quickstart.ipynb)

## Features

- **Differentiable Physics**: Compute gradients through entire FE simulations
- **High Performance**: JIT compilation and vectorization through JAX
- **GPU Acceleration**: Utilize Colab's free GPU resources

## Installation

First, install system dependencies required for gmsh (mesh generation):

In [ ]:
!apt update
!apt install -y libglu1 libxcursor-dev libxft2 libxinerama1 libfltk1.3-dev libfreetype6-dev libgl1-mesa-dev libocct-foundation-dev libocct-data-exchange-dev

Install FEAX:

In [ ]:
!pip install feax

## Verify Installation

Check that JAX and FEAX are properly installed:

In [1]:
import jax
import feax

print(f"JAX version: {jax.__version__}")
print(f"FEAX version: {feax.__version__}")
print(f"JAX devices: {jax.devices()}")
print(f"JAX 64-bit enabled: {jax.config.jax_enable_x64}")

JAX version: 0.5.3.dev20250415+209e0e405
FEAX version: 0.0.6.dev0
JAX devices: [CudaDevice(id=0)]
JAX 64-bit enabled: True


## Example: Create a Simple Mesh

Let's create a simple 3D box mesh using FEAX's mesh generation utilities:

In [2]:
from feax.mesh import box_mesh

# Create a 1m x 1m x 1m box with 0.2m element size
mesh = box_mesh(size=1.0, mesh_size=0.2, element_type='HEX8')

print(f"Mesh created successfully!")
print(f"Number of nodes: {mesh.points.shape[0]}")
print(f"Number of elements: {mesh.cells[0].data.shape[0]}")
print(f"Element type: HEX8 (8-node hexahedral)")

Mesh created successfully!
Number of nodes: 216
Number of elements: 8
Element type: HEX8 (8-node hexahedral)


## Example: Linear Elasticity Problem

Solve a simple linear elasticity problem on the mesh:

In [3]:
import jax.numpy as np
from feax import Problem, create_solver, SolverOptions
from feax import DirichletBC, DirichletBCSpec, InternalVars
from feax.utils import zero_like_initial_guess

# Define linear elasticity problem
class LinearElasticity(Problem):
    def get_tensor_map(self):
        def stress_tensor(u_grad, E, nu):
            # Linear elastic constitutive law
            mu = E / (2 * (1 + nu))
            lmbda = E * nu / ((1 + nu) * (1 - 2 * nu))
            epsilon = 0.5 * (u_grad + u_grad.T)
            return lmbda * np.trace(epsilon) * np.eye(self.dim) + 2 * mu * epsilon
        return stress_tensor

# Create problem
problem = LinearElasticity(mesh, vec=3, dim=3, ele_type='HEX8', gauss_order=2)
print("Problem created successfully!")

Problem created successfully!


In [ ]:
# Define boundary conditions (fix left face)
def left_boundary(point):
    return np.isclose(point[0], 0.0, atol=1e-5)

bc_specs = [
    DirichletBCSpec(left_boundary, 'all', 0.0)
]
bc = DirichletBC.from_specs(problem, bc_specs)

AttributeError: 'DirichletBC' object has no attribute 'dofs'

In [ ]:
# Set material properties (steel)
E = 210e9  # Young's modulus (Pa)
nu = 0.3   # Poisson's ratio

E_array = InternalVars.create_cell_var(problem, E)
nu_array = InternalVars.create_cell_var(problem, nu)
internal_vars = InternalVars(volume_vars=(E_array, nu_array), surface_vars=())
print("Material properties set: E = 210 GPa, ν = 0.3")

In [ ]:
# Configure and run solver
solver_options = SolverOptions(
    tol=1e-8,
    linear_solver="bicgstab",
    use_jacobi_preconditioner=False
)

solver = create_solver(problem, bc, solver_options, iter_num=1)
initial_guess = zero_like_initial_guess(problem, bc)

print("Solving...")
solution = solver(internal_vars, initial_guess)
print(f"Solution computed! Max displacement: {np.max(np.abs(solution)):.6e} m")

## Visualization

Save the solution to VTK format (download and open in ParaView):

In [ ]:
from feax.utils import save_sol

save_sol(mesh, "elasticity_solution.vtu",
         point_infos=[("displacement", solution.reshape(-1, 3))])

print("Solution saved to 'elasticity_solution.vtu'")
print("Download this file and open it in ParaView to visualize the results.")

## Next Steps

Explore more features of FEAX:

- **Different mesh types**: Try `sphere_mesh()`, `cylinder_mesh()`, `rectangle_mesh()`
- **Material gradients**: Use spatially varying material properties
- **Topology optimization**: Optimize material distribution
- **Automatic differentiation**: Compute gradients through the solver

### Documentation

- [FEAX Documentation](https://naruki-ichihara.github.io/feax/)
- [GitHub Repository](https://github.com/Naruki-Ichihara/feax)
- [More Examples](https://github.com/Naruki-Ichihara/feax/tree/main/examples)